In [1]:
from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl

In [2]:
from __future__ import print_function
import torch
from torch.utils.data import DataLoader, random_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]

Gs,y = loadDataset('/home/ines/Documents/M2/Stage/stage_ged/Ines/DeepGED/MAO/dataset.ds')
#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[1]
G2 = Gs[9]
print(y[1],y[9])

'''
plt.figure(0)
nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')

plt.figure(1)
nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')

plt.show()
'''

import extended_label
for g in Gs:
    extended_label.compute_extended_labels(g)
#for v in Gs[10].nodes():
#        print(Gs[10].nodes[v])

#print(nx.to_dict_of_lists(Gs[13]))



#dict={'C':0,'N':1,'O':2}
#A,labels=from_networkx_to_tensor2(Gs[13],dict)
#print(A)
#A1=(A==torch.ones(13,13)).int()
#A2=(A==2*torch.ones(13,13)).int()
#print(A1)
#print(A2)


edge max label 3
0.0 0.0


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import svd
import rings
from svd import iterated_power as compute_major_axis

#torch.autograd.set_detect_anomaly(True)

class Net(nn.Module):
    
        
    def __init__(self,GraphList,normalize=False,node_label='label'):
        super(Net, self).__init__()   
        self.normalize=normalize
        self.node_label=node_label
        dict,self.nb_edge_labels=self.build_node_dictionnary(GraphList)
        self.nb_labels=len(dict)
        print(self.nb_edge_labels)
        self.device='cuda' if torch.cuda.is_available() else 'cpu'
        nb_node_pair_label=self.nb_labels*(self.nb_labels-1)/2.0
        nb_edge_pair_label=int(self.nb_edge_labels*(self.nb_edge_labels-1)/2)
        
        self.node_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(int(self.nb_labels*(self.nb_labels-1)/2+1),requires_grad=True,device=self.device)) # all substitution costs+ nodeIns/Del. old version: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del        
        self.edge_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(nb_edge_pair_label+1,requires_grad=True,device=self.device)) #edgeIns/Del
        
        self.card=torch.tensor([G.order() for G in GraphList]).to(self.device)
        card_max=self.card.max()
        self.A=torch.empty((len(GraphList),card_max*card_max),dtype=torch.int,device=self.device)
        self.labels=torch.empty((len(GraphList),card_max),dtype=torch.int,device=self.device)
        print(self.A.shape)
        for k in range(len(GraphList)):
            A,l=self.from_networkx_to_tensor(GraphList[k],dict)             
            self.A[k,0:A.shape[1]]=A[0]
            self.labels[k,0:l.shape[0]]=l
        print('adjacency matrices',self.A)
        print('node labels',self.labels)
        print('order of the graphs',self.card)
        
    def forward(self, input):        
        ged=torch.zeros(len(input)).to(self.device) 
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()
        
            
        
        #print('weighs:',self.weighs.device,'device:',self.device,'card:',self.card.device,'A:',self.A.device,'labels:',self.labels.device)
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[k][1]
            n=self.card[g1]
            m=self.card[g2]
            
            self.ring_g,self.ring_h = rings.build_rings(g1,edgeInsDel.size()), rings.build_rings(g2,edgeInsDel.size()) 
            
            C=self.construct_cost_matrix(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)      
            #S=self.mapping_from_similarity(C,n,m)
            #S=self.mapping_from_cost(C,n,m)   
            #S=self.new_mapping_from_cost(C,n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
            S=self.mapping_from_cost_sans_FW(n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
            
            v=torch.flatten(S)
            
            normalize_factor=1.0
            if self.normalize:
                nb_edge1=(self.A[g1][0:n*n] != torch.zeros(n*n,device=self.device)).int().sum()
                nb_edge2=(self.A[g2][0:m*m] != torch.zeros(m*m,device=self.device)).int().sum()
                normalize_factor=nodeInsDel*(n+m)+edgeInsDel*(nb_edge1+nb_edge2)
            c=torch.diag(C)
            D=C-torch.eye(C.shape[0],device=self.device)*c
            ged[k]=(.5*v.T@D@v+c.T@v)/normalize_factor
        max=torch.max(ged)
        min=torch.min(ged)
        ged=(ged-min)/(max-min)
        
        return ged
    
    def from_weighs_to_costs(self):
        
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        cn=self.node_weighs*self.node_weighs
        ce=self.edge_weighs*self.edge_weighs
        total_cost=cn.sum()+ce.sum()
        cn=cn/total_cost #/max
        ce=ce/total_cost
        edgeInsDel=ce[-1]

        node_costs=torch.zeros((self.nb_labels,self.nb_labels),device=self.device)
        upper_part=torch.triu_indices(node_costs.shape[0],node_costs.shape[1],offset=1,device=self.device)        
        node_costs[upper_part[0],upper_part[1]]=cn[0:-1]
        node_costs=node_costs+node_costs.T

        if self.nb_edge_labels>1:
            edge_costs=torch.zeros((self.nb_edge_labels,self.nb_edge_labels),device=self.device)
            upper_part=torch.triu_indices(edge_costs.shape[0],edge_costs.shape[1],offset=1,device=self.device)        
            edge_costs[upper_part[0],upper_part[1]]=ce[0:-1]
            edge_costs=edge_costs+edge_costs.T
        else:
            edge_costs=torch.zeros(0,device=self.device)
        
        return node_costs,cn[-1],edge_costs,edgeInsDel
    
    def build_node_dictionnary(self,GraphList):
        #extraction de tous les labels d'atomes
        node_labels=[]
        for G in Gs:
            for v in nx.nodes(G):
                if not G.nodes[v][self.node_label][0] in node_labels:
                    node_labels.append(G.nodes[v][self.node_label][0])
        node_labels.sort()
        #extraction d'un dictionnaire permettant de numéroter chaque label par un numéro.
        dict={}
        k=0
        for label in node_labels:
            dict[label]=k
            k=k+1
        print(node_labels)
        print(dict,len(dict))
    
        return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))
    
    def from_networkx_to_tensor(self,G,dict):    
        A=torch.tensor(nx.to_scipy_sparse_matrix(G,dtype=int,weight='bond_type').todense(),dtype=torch.int)        
        lab=[dict[G.nodes[v][self.node_label][0]] for v in nx.nodes(G)]
   
        return (A.view(1,A.shape[0]*A.shape[1]),torch.tensor(lab))

    def construct_cost_matrix(self,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel):
        n = self.card[g1].item()
        m = self.card[g2].item()
        
        A1=torch.zeros((n+1,n+1),dtype=torch.int,device=self.device)
        A1[0:n,0:n]=self.A[g1][0:n*n].view(n,n)
        A2=torch.zeros((m+1,m+1),dtype=torch.int,device=self.device)
        A2[0:m,0:m]=self.A[g2][0:m*m].view(m,m)
        
        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
        C=edgeInsDel*self.matrix_edgeInsDel(A1,A2)
        if self.nb_edge_labels>1:
            for k in range(self.nb_edge_labels):
                for l in range(self.nb_edge_labels):
                    if k != l:
#                    C.add_(self.matrix_edgeSubst(A1,A2,k+1,l+1),alpha=edge_costs[k][l])
                        C=C+edge_costs[k][l]*self.matrix_edgeSubst(A1,A2,k+1,l+1)
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        

        l1=self.labels[g1][0:n]
        l2=self.labels[g2][0:m]
        D=torch.zeros((n+1)*(m+1),device=self.device)
        D[n*(m+1):]=nodeInsDel
        D[n*(m+1)+m]=0
        D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
        D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([node_costs[l1[k//(m+1)],l2[k%(m+1)]] for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
        mask = torch.diag(torch.ones_like(D))
        C=mask*torch.diag(D) + (1. - mask)*C
        
        #C[range(len(C)),range(len(C))]=D
      
        return C
    def matrix_edgeInsDel(self,A1,A2):
        Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1]),device=self.device))
        Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1]),device=self.device))
        C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
        C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
        C12=torch.logical_or(C1,C2).int()
    
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)

    def matrix_edgeSubst(self,A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]),device=self.device)).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]),device=self.device)).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1)
    
    def similarity_from_cost(self,C):
        N=C.shape[0]
             
        #return (torch.norm(C,p='fro')*torch.eye(N,device=self.device) -C)
        return (C.max()*torch.eye(N,device=self.device) -C)
    
    def lsape_populate_instance(self,first_graph,second_graph,average_node_cost, average_edge_cost,alpha,lbda):       #ring_g, ring_h come from global ring with all graphs in so ring_g = rings['g'] and ring_h = rings['h']
        g,h = Gs[first_graph], Gs[second_graph]
        self.average_cost =[average_node_cost, average_edge_cost]
        self.first_graph, self.second_graph = first_graph,second_graph
        
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()

        lsape_instance = [[0 for _ in range(len(g) + 1)] for __ in range(len(h) + 1)]
        for g_node_index in range(len(g) + 1):
            for h_node_index in range(len(h) + 1):
                lsape_instance[h_node_index][g_node_index] = rings.compute_ring_distance(g,h,self.ring_g,self.ring_h,g_node_index,h_node_index,alpha,lbda,node_costs,nodeInsDel,edge_costs,edgeInsDel,first_graph,second_graph)
        for i in lsape_instance :
            i = torch.as_tensor(i)
        lsape_instance = torch.as_tensor(lsape_instance)
        #print(type(lsape_instance))
        return lsape_instance
    
  
    def mapping_from_cost_sans_FW(self,n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel): 
        c_0 =self.lsape_populate_instance(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
        x0=svd.eps_assigment_from_mapping(torch.exp(-c_0),10).view((n+1)*(m+1),1)
        return x0
    
    def new_mapping_from_cost(self,C,n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel): 
        c=torch.diag(C)       
        c_0 =self.lsape_populate_instance(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=svd.eps_assigment_from_mapping(torch.exp(-c_0),10).view((n+1)*(m+1),1)
        return svd.franck_wolfe(x0,D,c,5,15,n,m)
    
    
    def mapping_from_cost(self,C,n,m):
        c=torch.diag(C)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=svd.eps_assigment_from_mapping(torch.exp(-.5*c.view(n+1,m+1)),10).view((n+1)*(m+1),1)
        x=svd.franck_wolfe(x0,D,c,5,10,n,m)
        def print_grad(grad):
            if(grad.norm()!= 0.0):
                print(grad)
        
#        x.register_hook(print_grad)
        return x

print('Gs=',len(Gs))
model = Net(Gs,normalize=True,node_label='extended_label')

params = list(model.parameters())
print(len(params))
print(params[0])
#print(model(input))
print(max([G.order() for G in Gs]),len(Gs))
print('toto')

Gs= 68
['C_1C', 'C_1C1C1N', 'C_1C1C2C', 'C_1C1N', 'C_1C1N2C', 'C_1C1O', 'C_1C1O2C', 'C_1C2C', 'C_1C3C', 'C_1N', 'C_1O', 'C_2C', 'C_2C2C', 'C_3C', 'N_1C', 'N_1C1C', 'N_1C1C1C', 'O_1C', 'O_1C1C']
{'C_1C': 0, 'C_1C1C1N': 1, 'C_1C1C2C': 2, 'C_1C1N': 3, 'C_1C1N2C': 4, 'C_1C1O': 5, 'C_1C1O2C': 6, 'C_1C2C': 7, 'C_1C3C': 8, 'C_1N': 9, 'C_1O': 10, 'C_2C': 11, 'C_2C2C': 12, 'C_3C': 13, 'N_1C': 14, 'N_1C1C': 15, 'N_1C1C1C': 16, 'O_1C': 17, 'O_1C1C': 18} 19
3
torch.Size([68, 729])
adjacency matrices tensor([[         0,          1,          0,  ..., 1600481124, 1953722211,
         1869491315],
        [         0,          1,          0,  ...,  694969668,  538970682,
          538976288],
        [         0,          1,          0,  ...,  540876903, 1735289202,
         1730632563],
        ...,
        [         0,          1,          0,  ...,  778464357, 1685217635,
          538970665],
        [         0,          1,          0,  ..., 1701585011, 1634476147,
         1936483682],
        [

In [5]:
import itertools

nb=len(Gs)
class1=torch.tensor([k for k in range(len(y)) if y[k]==1])
class2=torch.tensor([k for k in range(len(y)) if y[k]==0])

nb_class1=12
nb_class2=int((nb_class1-1)/2)
train_size=nb_class1+nb_class2
#train_size=20

#if train_size % 2 == 0:
#    nb_class1=int(train_size/2)
#    nb_class2=int(train_size/2)
#else:
#    nb_class1=int(train_size/2)+1
#    nb_class2=int(train_size/2)
    
print((torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long())
random_class1=class1[(torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long()]
random_class2=class2[(torch.abs(10000*torch.randn(nb_class2)).int()%class2.size()[0]).long()]
train_graphs=torch.cat((random_class1,random_class2),0)
print('train graphs:',train_graphs)


couples=torch.triu_indices(train_size,train_size,offset=1)
print('couples=',couples)
print('nb_class1/nb_class2=',nb_class1,nb_class2)
#combinations=itertools.combinations(range(nb),2)

nb_elt=int(nb_class1*(nb_class1+2*nb_class2-1)/2)
print('couples restreints:',couples[:,0:nb_elt])

#nb_elt=int(train_size*(train_size-1)/2)
data=torch.empty((nb_elt,2),dtype=torch.int)
yt=torch.ones(nb_elt)
print('old_size, new size=',nb_elt,.5*nb_class1*(nb_class1+2*nb_class2-1))
data[0:nb_elt,0]=train_graphs[couples[0,0:nb_elt]]
data[0:nb_elt,1]=train_graphs[couples[1,0:nb_elt]]


#data[0:nb_elt,0]=train_graphs[couples[0]]
#data[0:nb_elt,1]=train_graphs[couples[1]]
print(nb_elt)
#couples=[]
for k in range(nb_elt):
    if (y[data[k][0]]!=y[data[k][1]]):
        yt[k]=-1.0        

print('data=',data)

#print(couples[1:2])

torch.cuda.empty_cache()
if torch.cuda.is_available():
    device = torch.device("cuda:0")          # a CUDA device object    


tensor([ 4,  3, 25, 11, 21,  2, 11, 25, 14, 17, 17, 12])
train graphs: tensor([19, 37, 59, 34, 18, 10, 19, 10, 53, 60, 60, 25,  6, 12, 61, 42, 49])
couples= tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,
          5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,
          6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,
          9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11,
         12, 12, 12, 12, 13, 13, 13, 14, 14, 15],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  2,  3,
          4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  3,  4,  5,  6,  7,
          8,  9, 10, 11, 12, 13, 14, 15, 16,  4,  5,  6,  7,  8,

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from torch.utils.data import DataLoader, random_split, TensorDataset

[train_D, valid_D,train_L,valid_L]= train_test_split(data,yt, test_size=0.25,train_size=0.75, shuffle=True) #, stratify=yt)
#print(train_D)   
    
DatasetTrain = TensorDataset(train_D, train_L)

DatasetValid=TensorDataset(valid_D, valid_L)

trainloader=torch.utils.data.DataLoader(DatasetTrain,batch_size=len(train_D),shuffle=True,drop_last=True, num_workers=0)
#len(train_D) #31
validationloader=torch.utils.data.DataLoader(DatasetValid, batch_size=8, drop_last=True,num_workers=0)

print(len(train_D), len(valid_D))
print(len(trainloader),len(validationloader))
for i,j in validationloader:
    print('i : ',i,'\n j : ',j)

94 32
1 4
i :  tensor([[19, 59],
        [18, 25],
        [19, 61],
        [25, 42],
        [37, 53],
        [59, 12],
        [19, 60],
        [10, 42]], dtype=torch.int32) 
 j :  tensor([ 1.,  1., -1., -1.,  1., -1.,  1., -1.])
i :  tensor([[34, 60],
        [19, 53],
        [18, 12],
        [18, 42],
        [59, 10],
        [37, 42],
        [19, 42],
        [60, 49]], dtype=torch.int32) 
 j :  tensor([ 1.,  1., -1., -1.,  1., -1., -1., -1.])
i :  tensor([[59, 49],
        [53, 49],
        [19, 18],
        [60, 49],
        [19, 25],
        [60,  6],
        [19, 60],
        [53, 60]], dtype=torch.int32) 
 j :  tensor([-1., -1.,  1., -1.,  1., -1.,  1.,  1.])
i :  tensor([[60, 60],
        [59,  6],
        [60, 25],
        [18, 19],
        [59, 18],
        [59, 61],
        [19, 10],
        [37, 10]], dtype=torch.int32) 
 j :  tensor([ 1., -1.,  1.,  1.,  1., -1.,  1.,  1.])


In [7]:
#if torch.cuda.device_count() > 1:
#  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#  model = nn.DataParallel(model)
from triangular_losses import TriangularConstraint as triangular_constraint

model.to(device)
def classification(model,data,yt,nb_iter):

    criterion = torch.nn.HingeEmbeddingLoss(margin=1.0,reduction='mean')
    criterionTri=triangular_constraint()
    optimizer = torch.optim.Adam(model.parameters()) #, lr=1e-3
    #print(device)

    #torch.cat((same_class[0:20],diff_class[0:20]),0).to(device)
    whole_input=data.to(device)
    target=yt.to(device) 
    #torch.ones(40,device=device)
    #target[20:]=-1.0
    #target=(yt[0:20]).to(device)
    InsDel=np.empty((nb_iter,2))
    node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
    nodeSub=np.empty((nb_iter,int(node_costs.shape[0]*(node_costs.shape[0]-1)/2)))
    edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
    loss_plt=np.empty(nb_iter)
    loss_train_plt=np.empty(nb_iter)
    loss_valid_plt=np.empty(nb_iter)
    min_valid_loss = np.inf
    iter_min_valid_loss = 0
    
    for t in range(nb_iter):    
        train_loss = 0.0
        valid_loss = 0.0
        tmp=np.inf
        for train_data,train_labels in trainloader:
            optimizer.zero_grad()
            
            inputt=train_data.to(device)
            
            # Forward pass: Compute predicted y by passing data to the model
            y_pred = model(whole_input).to(device)

            # Compute and print loss
            loss = criterion(y_pred, target).to(device)
            node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
            triangularInq=criterionTri(node_costs,nodeInsDel,edge_costs,edgeInsDel)
            loss=loss*(1+triangularInq)
            loss.to(device)
            # Zero gradients, perform a backward pass, and update the weights.
            #optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            print('loss.item of the train = ', t, loss.item())
            train_loss =+ loss.item() #* train_data.size(0) 
            if (loss.item()<tmp): tmp=loss.item()
        loss_plt[t]=loss.item()  
        loss_train_plt[t]=train_loss /len(trainloader)
        #loss_plt[t]=tmp
            
        if t % 100 == 99 or t==0:   
            print('ged=',y_pred*target)  #train_labels
            print('Distances: ',y_pred)
            print('Loss Triangular:',triangularInq.item())
            print('node_costs :')
            print(node_costs)
            print('nodeInsDel:',nodeInsDel.item())
            print('edge_costs :')
            print(edge_costs)
            print('edgeInsDel:',edgeInsDel.item())
            
            
            
        for valid_data,valid_labels in validationloader:
            
            inputt=valid_data.to(device)
            y_pred = model(inputt).to(device)
            # Compute and print loss
            loss = criterion(y_pred, valid_labels).to(device)    
            loss.to(device)
            
            print('loss.item of the valid = ', t, loss.item())  
            valid_loss = valid_loss + loss.item() #* valid_data.size(0)
            
        loss_valid_plt[t]=valid_loss / len(validationloader)   
        
        InsDel[t][0]=nodeInsDel.item()
        InsDel[t][1]=edgeInsDel.item()
        
        
        k=0
        for p in range(node_costs.shape[0]):
            for q in range(p+1,node_costs.shape[0]):
                nodeSub[t][k]=node_costs[p][q]
                k=k+1
        k=0
        for p in range(edge_costs.shape[0]):
            for q in range(p+1,edge_costs.shape[0]):
                edgeSub[t][k]=edge_costs[p][q]
                k=k+1
        
            
        print(f'Iteration {t+1} \t\t Training Loss: {\
        train_loss / len(trainloader)} \t\t Validation Loss: {\
        valid_loss / len(validationloader)}')
        if min_valid_loss > valid_loss:
            print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f})')
            min_valid_loss = valid_loss
            iter_min_valid_loss = t
            nodeSub_min = node_costs
            edgeSub_min = edge_costs
            nodeInsDel_min = nodeInsDel
            edgeInsDel_min = edgeInsDel
            
            
    print('iter and min_valid_loss = ',iter_min_valid_loss, min_valid_loss)
    '''
    nodeInsDel_min = InsDel[iter_min_valid_loss][0]
    edgeInsDel_min = InsDel[iter_min_valid_loss][1]
    nodeSub_min = nodeSub[iter_min_valid_loss]
    edgeSub_min = edgeSub[iter_min_valid_loss]
    '''
    print(' Min cost for nodeInsDel = ', nodeInsDel_min)
    print(' Min cost for edgeInsDel = ', edgeInsDel_min)
    print(' Min cost for nodeSub = ', nodeSub_min)
    print(' Min cost for edgeSub = ', edgeSub_min)
    torch.save(nodeInsDel_min, 'nodeInsDel_min', pickle_module=pkl) 
    torch.save(edgeInsDel_min, 'edgeInsDel_min', pickle_module=pkl) 
    torch.save(nodeSub_min, 'nodeSub_min', pickle_module=pkl) 
    torch.save(edgeSub_min, 'edgeSub_min', pickle_module=pkl)
    return InsDel,nodeSub,edgeSub,loss_plt,loss_valid_plt,loss_train_plt



In [8]:
nb_iter=100
InsDel, nodeSub,edgeSub,loss_plt,loss_valid_plt,loss_train_plt=classification(model,data,yt,nb_iter)
plt.figure(0)
plt.plot(InsDel[0:nb_iter,0],label="node")
plt.plot(InsDel[0:nb_iter,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:nb_iter,k])
plt.title('Node Substitutions costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:nb_iter,k])
plt.title('Edge Substitutions costs')
plt.figure(3)
plt.plot(loss_plt)
plt.title('Evolution of the train loss (loss_plt)')

plt.figure(4)
plt.plot(loss_valid_plt)
plt.title('Evolution of the valid loss')
'''
plt.figure(5)
plt.plot(loss_train_plt)
plt.title('Evolution of the loss_train_plt')
'''
plt.show()
plt.close()

loss.item of the train =  0 0.3693350851535797
ged= tensor([ 0.3859,  0.3234,  0.2993,  0.2914,  0.0000,  0.0000,  0.0000,  0.2591,
         0.2987,  0.2987,  0.3743, -0.2228, -0.2621, -0.4488, -0.9263, -0.6886,
         0.2535,  0.2650,  0.2336,  0.2825,  0.2740,  0.2825,  0.3091,  0.2861,
         0.2861,  0.0521, -0.3068, -0.5058, -0.3355, -0.7366, -0.5054,  0.0587,
         0.0432,  0.2721,  0.2722,  0.2721,  0.2680,  0.0563,  0.0563,  0.3374,
        -0.2626, -0.4402, -0.3470, -0.7345, -0.5087,  0.0530,  0.2511,  0.2512,
         0.2511,  0.2687,  0.0458,  0.0458,  0.3409, -0.2487, -0.4226, -0.3225,
        -0.7342, -0.4725,  0.2211,  0.2211,  0.2211,  0.2766,  0.0438,  0.0438,
         0.3244, -0.2461, -0.3732, -0.2932, -0.6886, -0.4946,  0.0000,  0.0000,
         0.2670,  0.2963,  0.2963,  0.3765, -0.2308, -0.2716, -0.4547, -0.9405,
        -0.6854,  0.0000,  0.2591,  0.2987,  0.2987,  0.3743, -0.2228, -0.2621,
        -0.4488, -0.9263, -0.6886,  0.2670,  0.2963,  0.2963,  0.376

loss.item of the train =  6 0.32140669226646423
loss.item of the valid =  6 0.2345772236585617
loss.item of the valid =  6 0.2439938336610794
loss.item of the valid =  6 0.26560238003730774
loss.item of the valid =  6 0.23751287162303925
Iteration 7 		 Training Loss: 0.32140669226646423 		 Validation Loss: 0.24542157724499702
loss.item of the train =  7 0.3214847445487976
loss.item of the valid =  7 0.23568563163280487
loss.item of the valid =  7 0.24272069334983826
loss.item of the valid =  7 0.26342421770095825
loss.item of the valid =  7 0.24614295363426208
Iteration 8 		 Training Loss: 0.3214847445487976 		 Validation Loss: 0.24699337407946587
loss.item of the train =  8 0.3220807909965515
loss.item of the valid =  8 0.2292618602514267
loss.item of the valid =  8 0.2421858012676239
loss.item of the valid =  8 0.26907065510749817
loss.item of the valid =  8 0.26681941747665405
Iteration 9 		 Training Loss: 0.3220807909965515 		 Validation Loss: 0.2518344335258007
loss.item of the tr

loss.item of the valid =  30 0.2781673073768616
loss.item of the valid =  30 0.28153467178344727
Iteration 31 		 Training Loss: 0.3182224929332733 		 Validation Loss: 0.25796404108405113
loss.item of the train =  31 0.31897082924842834
loss.item of the valid =  31 0.22655630111694336
loss.item of the valid =  31 0.24608729779720306
loss.item of the valid =  31 0.2783959209918976
loss.item of the valid =  31 0.2710650861263275
Iteration 32 		 Training Loss: 0.31897082924842834 		 Validation Loss: 0.2555261515080929
loss.item of the train =  32 0.3185766339302063
loss.item of the valid =  32 0.22717776894569397
loss.item of the valid =  32 0.2475024163722992
loss.item of the valid =  32 0.2795235812664032
loss.item of the valid =  32 0.2227213978767395
Iteration 33 		 Training Loss: 0.3185766339302063 		 Validation Loss: 0.24423129111528397
loss.item of the train =  33 0.3192358613014221
loss.item of the valid =  33 0.22719456255435944
loss.item of the valid =  33 0.24740086495876312
los

loss.item of the valid =  55 0.2275598794221878
loss.item of the valid =  55 0.23963981866836548
loss.item of the valid =  55 0.2694678008556366
loss.item of the valid =  55 0.3716253638267517
Iteration 56 		 Training Loss: 0.3201787769794464 		 Validation Loss: 0.2770732156932354
loss.item of the train =  56 0.3214499056339264
loss.item of the valid =  56 0.2275913655757904
loss.item of the valid =  56 0.2381618618965149
loss.item of the valid =  56 0.26744773983955383
loss.item of the valid =  56 0.37862592935562134
Iteration 57 		 Training Loss: 0.3214499056339264 		 Validation Loss: 0.2779567241668701
loss.item of the train =  57 0.32157212495803833
loss.item of the valid =  57 0.22800977528095245
loss.item of the valid =  57 0.23780497908592224
loss.item of the valid =  57 0.2661355435848236
loss.item of the valid =  57 0.36504051089286804
Iteration 58 		 Training Loss: 0.32157212495803833 		 Validation Loss: 0.2742477022111416
loss.item of the train =  58 0.32103368639945984
loss

loss.item of the train =  80 0.3225846588611603
loss.item of the valid =  80 0.2310691624879837
loss.item of the valid =  80 0.23435743153095245
loss.item of the valid =  80 0.2582382261753082
loss.item of the valid =  80 0.3881944417953491
Iteration 81 		 Training Loss: 0.3225846588611603 		 Validation Loss: 0.2779648154973984
loss.item of the train =  81 0.32179051637649536
loss.item of the valid =  81 0.23001383244991302
loss.item of the valid =  81 0.2329050600528717
loss.item of the valid =  81 0.258402943611145
loss.item of the valid =  81 0.4097956717014313
Iteration 82 		 Training Loss: 0.32179051637649536 		 Validation Loss: 0.28277937695384026
loss.item of the train =  82 0.3219616115093231
loss.item of the valid =  82 0.22963733971118927
loss.item of the valid =  82 0.23192062973976135
loss.item of the valid =  82 0.2584697902202606
loss.item of the valid =  82 0.4272352457046509
Iteration 83 		 Training Loss: 0.3219616115093231 		 Validation Loss: 0.28681575134396553
loss.i

loss.item of the valid =  99 0.2315569967031479
loss.item of the valid =  99 0.23747369647026062
loss.item of the valid =  99 0.2606603503227234
loss.item of the valid =  99 0.37044501304626465
Iteration 100 		 Training Loss: 0.32152414321899414 		 Validation Loss: 0.27503401413559914
iter and min_valid_loss =  25 0.9715203642845154
 Min cost for nodeInsDel =  tensor(0.0598, grad_fn=<SelectBackward>)
 Min cost for edgeInsDel =  tensor(0.0331, grad_fn=<SelectBackward>)
 Min cost for nodeSub =  tensor([[0.0000, 0.0048, 0.0053, 0.0048, 0.0053, 0.0052, 0.0048, 0.0054, 0.0051,
         0.0055, 0.0050, 0.0054, 0.0048, 0.0055, 0.0051, 0.0045, 0.0046, 0.0059,
         0.0060],
        [0.0048, 0.0000, 0.0059, 0.0054, 0.0057, 0.0054, 0.0060, 0.0045, 0.0059,
         0.0056, 0.0047, 0.0051, 0.0060, 0.0057, 0.0047, 0.0059, 0.0044, 0.0054,
         0.0055],
        [0.0053, 0.0059, 0.0000, 0.0058, 0.0046, 0.0056, 0.0055, 0.0056, 0.0058,
         0.0046, 0.0055, 0.0046, 0.0049, 0.0049, 0.0051, 0.00

In [9]:
A=torch.tensor(nx.to_scipy_sparse_matrix(Gs[0],dtype=int,weight='bond_type').todense(),dtype=torch.int) 
print(A)

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0],
        [1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]], dtype=torch.int32)


In [10]:
plt.plot(loss_plt, label='train loss')
plt.plot(loss_valid_plt, label='valid loss')
plt.title('Train and valid losses')
plt.legend()
plt.show()

In [11]:
plt.figure(0)
plt.plot(InsDel[0:500,0],label="node")
plt.plot(InsDel[0:500,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:500,k])
plt.title('node Substitution costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:500,k])
plt.title('edge Substitution costs')
plt.show()
plt.close()